# Прост класификатор на изображения

Този ноутбук показва как да класифицирате изображения, използвайки предварително обучена невронна мрежа.

**Какво ще научите:**
- Как да заредите и използвате предварително обучен модел
- Предварителна обработка на изображения
- Правене на прогнози върху изображения
- Разбиране на оценките за увереност

**Приложение:** Идентифициране на обекти в изображения (като "котка", "куче", "кола" и т.н.)

---


## Стъпка 1: Импортиране на необходимите библиотеки

Нека импортираме инструментите, които са ни нужни. Не се притеснявайте, ако все още не разбирате всички тях!


In [ ]:
# Core libraries
import numpy as np
from PIL import Image
import requests
from io import BytesIO

# TensorFlow for deep learning
try:
    import tensorflow as tf
    from tensorflow.keras.applications import MobileNetV2
    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
    print("✅ TensorFlow loaded successfully!")
    print(f"   Version: {tf.__version__}")
except ImportError:
    print("❌ Please install TensorFlow: pip install tensorflow")

## Стъпка 2: Зареждане на предварително обучен модел

Ще използваме **MobileNetV2**, невронна мрежа, която вече е обучена върху милиони изображения.

Това се нарича **Трансферно обучение** - използване на модел, който някой друг е обучил!


In [ ]:
print("📦 Loading pre-trained MobileNetV2 model...")
print("   This may take a minute on first run (downloading weights)...")

# Load the model
# include_top=True means we use the classification layer
# weights='imagenet' means it was trained on ImageNet dataset
model = MobileNetV2(weights='imagenet', include_top=True)

print("✅ Model loaded!")
print(f"   The model can recognize 1000 different object categories")

## Стъпка 3: Помощни функции

Нека създадем функции за зареждане и подготовка на изображения за нашия модел.


In [ ]:
def load_image_from_url(url):
    """
    Load an image from a URL.
    
    Args:
        url: Web address of the image
        
    Returns:
        PIL Image object
    """
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img


def prepare_image(img):
    """
    Prepare an image for the model.
    
    Steps:
    1. Resize to 224x224 (model's expected size)
    2. Convert to array
    3. Add batch dimension
    4. Preprocess for MobileNetV2
    
    Args:
        img: PIL Image
        
    Returns:
        Preprocessed image array
    """
    # Resize to 224x224 pixels
    img = img.resize((224, 224))
    
    # Convert to numpy array
    img_array = np.array(img)
    
    # Add batch dimension (model expects multiple images)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Preprocess for MobileNetV2
    img_array = preprocess_input(img_array)
    
    return img_array


def classify_image(img):
    """
    Classify an image and return top predictions.
    
    Args:
        img: PIL Image
        
    Returns:
        List of (class_name, confidence) tuples
    """
    # Prepare the image
    img_array = prepare_image(img)
    
    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    
    # Decode predictions to human-readable labels
    # top=5 means we get the top 5 most likely classes
    decoded = decode_predictions(predictions, top=5)[0]
    
    # Convert to simpler format
    results = [(label, float(confidence)) for (_, label, confidence) in decoded]
    
    return results


print("✅ Helper functions ready!")

## Стъпка 4: Тест върху примерни изображения

Нека опитаме да класифицираме някои изображения от интернет!


In [ ]:
# Sample images to classify
# These are from Unsplash (free stock photos)
test_images = [
    {
        "url": "https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba?w=400",
        "description": "A cat"
    },
    {
        "url": "https://images.unsplash.com/photo-1552053831-71594a27632d?w=400",
        "description": "A dog"
    },
    {
        "url": "https://images.unsplash.com/photo-1511919884226-fd3cad34687c?w=400",
        "description": "A car"
    },
]

print(f"🧪 Testing on {len(test_images)} images...")
print("=" * 70)

### Класифицирайте всяко изображение


In [ ]:
for i, img_data in enumerate(test_images, 1):
    print(f"\n📸 Image {i}: {img_data['description']}")
    print("-" * 70)
    
    try:
        # Load image
        img = load_image_from_url(img_data['url'])
        
        # Display image
        display(img.resize((200, 200)))  # Show smaller version
        
        # Classify
        results = classify_image(img)
        
        # Show predictions
        print("\n🎯 Top 5 Predictions:")
        for rank, (label, confidence) in enumerate(results, 1):
            # Create a visual bar
            bar_length = int(confidence * 50)
            bar = "█" * bar_length
            
            print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
        
    except Exception as e:
        print(f"❌ Error: {e}")

print("\n" + "=" * 70)

## Стъпка 5: Опитайте със свои изображения!

Заменете URL адреса по-долу с който и да е URL адрес на изображение, което искате да класифицирате.


In [ ]:
# Try your own image!
# Replace this URL with any image URL
custom_image_url = "https://images.unsplash.com/photo-1472491235688-bdc81a63246e?w=400"  # A flower

print("🖼️  Classifying your custom image...")
print("=" * 70)

try:
    # Load and show image
    img = load_image_from_url(custom_image_url)
    display(img.resize((300, 300)))
    
    # Classify
    results = classify_image(img)
    
    # Show results
    print("\n🎯 Top 5 Predictions:")
    print("-" * 70)
    for rank, (label, confidence) in enumerate(results, 1):
        bar_length = int(confidence * 50)
        bar = "█" * bar_length
        print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
    
    # Highlight top prediction
    top_label, top_confidence = results[0]
    print("\n" + "=" * 70)
    print(f"\n🏆 Best guess: {top_label} ({top_confidence*100:.2f}% confident)")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("   Make sure the URL points to a valid image!")

## 💡 Какво се случи?

1. **Заредихме предварително обучен модел** - MobileNetV2 е обучен върху милиони изображения
2. **Предварително обработихме изображенията** - Преоразмерихме и форматирахме ги за модела
3. **Моделът направи прогнози** - Той изведе вероятности за 1000 категории обекти
4. **Декодирахме резултатите** - Превърнахме числата в разбираеми за хората етикети

### Разбиране на оценките за увереност

- **90-100%**: Много уверен (почти сигурно правилно)
- **70-90%**: Уверен (вероятно правилно)
- **50-70%**: Донякъде уверен (може би правилно)
- **Под 50%**: Не много уверен (несигурно)

### Защо прогнозите може да са грешни?

- **Необичаен ъгъл или осветление** - Моделът е обучен върху типични снимки
- **Множество обекти** - Моделът очаква един основен обект
- **Редки обекти** - Моделът познава само 1000 категории
- **Ниско качество на изображението** - Размазаните или пикселизирани изображения са по-трудни


## 🚀 Следващи стъпки

1. **Опитайте с различни изображения:**
   - Намерете изображения на [Unsplash](https://unsplash.com)
   - Десен бутон → „Копирай адреса на изображението“, за да получите URL

2. **Експериментирайте:**
   - Какво се случва с абстрактно изкуство?
   - Може ли да разпознава обекти от различни ъгли?
   - Как се справя с множество обекти?

3. **Научете повече:**
   - Разгледайте [уроците за компютърно зрение](../lessons/4-ComputerVision/README.md)
   - Научете как да обучите собствен класификатор на изображения
   - Разберете как работят CNNs (Конволюционни невронни мрежи)

---

## 🎉 Поздравления!

Току-що създадохте класификатор на изображения, използвайки съвременна невронна мрежа!

Същата техника се използва за:
- Google Photos (организиране на вашите снимки)
- Автономни автомобили (разпознаване на обекти)
- Медицинска диагностика (анализ на рентгенови снимки)
- Контрол на качеството (откриване на дефекти)

Продължавайте да изследвате и учите! 🚀



---

**Отказ от отговорност**:  
Този документ е преведен с помощта на AI услуга за превод [Co-op Translator](https://github.com/Azure/co-op-translator). Въпреки че се стремим към точност, моля, имайте предвид, че автоматизираните преводи може да съдържат грешки или неточности. Оригиналният документ на неговия роден език трябва да се счита за авторитетен източник. За критична информация се препоръчва професионален човешки превод. Ние не носим отговорност за недоразумения или погрешни интерпретации, произтичащи от използването на този превод.
